In [10]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dense, Lambda, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# training data
train_x_path = 'coding_test/train_x.npy'  
train_y_path = 'coding_test/train_y.npy'  
train_x = np.load(train_x_path)
train_y = np.load(train_y_path)

# function for Capsule Network
def squash(x, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(x), axis, keepdims=True) + tf.keras.backend.epsilon()
    scale = tf.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x

# PrimaryCaps layer
def PrimaryCaps(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    output = Conv2D(filters=dim_capsule * n_channels, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
    outputs = Reshape(target_shape=[-1, dim_capsule])(output)
    return Lambda(squash)(outputs)

# Capsule Layer
def CapsuleLayer(num_capsule, dim_capsule, routings=3):
    def layer(inputs):
        input_shape = tf.shape(inputs)
        num_input_capsule = input_shape[1]
        inputs_expand = tf.expand_dims(inputs, 1)
        inputs_tiled = tf.tile(inputs_expand, [1, num_capsule, 1, 1])
        b = tf.zeros(shape=[num_capsule, num_input_capsule])

        for i in range(routings):
            c = tf.nn.softmax(b, axis=1)
            outputs = squash(tf.linalg.matmul(c, inputs_tiled))
            if i < routings - 1:
                b += tf.linalg.matmul(outputs, inputs_tiled, transpose_b=True)

        return outputs
    return layer
# Model 
input_shape = (23, 3)
inputs = Input(shape=input_shape)
reshaped_inputs = Reshape((23, 3, 1))(inputs)
primary_caps = PrimaryCaps(reshaped_inputs, dim_capsule=8, n_channels=16, kernel_size=(3, 3), strides=1, padding='valid')
caps_output = CapsuleLayer(num_capsule=31, dim_capsule=4)(primary_caps)
flatten_output = Flatten()(caps_output)
# Adjusted output layer to match the shape of train_y
output = Dense(31*4, activation='linear')(flatten_output)
reshaped_output = Reshape((31, 4))(output)  # Reshape to match the target shape
model = Model(inputs=inputs, outputs=reshaped_output)  # Corrected line

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=10, batch_size=32, validation_split=0.2)

model.save('capsule_network_model.h5')

Epoch 1/10
145/145 [==============================] - 46s 306ms/step - loss: 0.0694 - accuracy: 0.9327 - val_loss: 0.0260 - val_accuracy: 0.9396
Epoch 2/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0190 - accuracy: 0.9585 - val_loss: 0.0269 - val_accuracy: 0.9398
Epoch 3/10
145/145 [==============================] - 43s 299ms/step - loss: 0.0188 - accuracy: 0.9585 - val_loss: 0.0263 - val_accuracy: 0.9441
Epoch 4/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0194 - accuracy: 0.9579 - val_loss: 0.0269 - val_accuracy: 0.9397
Epoch 5/10
145/145 [==============================] - 44s 304ms/step - loss: 0.0192 - accuracy: 0.9579 - val_loss: 0.0300 - val_accuracy: 0.9396
Epoch 6/10
145/145 [==============================] - 44s 300ms/step - loss: 0.0193 - accuracy: 0.9586 - val_loss: 0.0286 - val_accuracy: 0.9363
Epoch 7/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0194 - accuracy: 0.9578 - val_loss: 0.0279 - val_ac

In [ ]:
#**Data Preprocessing**

In [9]:
import numpy as np

train_x = np.load('coding_test/train_x.npy') 
train_y = np.load('coding_test/train_y.npy') 

train_x = train_x / np.max(train_x)
train_y = train_y / np.max(train_y)

In [ ]:
#Building the Capsule Network Model

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dense, Lambda, Flatten
from tensorflow.keras.models import Model

def squash(x, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(x), axis, keepdims=True) + tf.keras.backend.epsilon()
    scale = tf.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x

def PrimaryCaps(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    output = Conv2D(filters=dim_capsule * n_channels, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
    outputs = Reshape(target_shape=[-1, dim_capsule])(output)
    return Lambda(squash)(outputs)

def CapsuleLayer(num_capsule, dim_capsule, routings=3):
    def layer(inputs):
        input_shape = tf.shape(inputs)
        num_input_capsule = input_shape[1]
        inputs_expand = tf.expand_dims(inputs, 1)
        inputs_tiled = tf.tile(inputs_expand, [1, num_capsule, 1, 1])
        b = tf.zeros(shape=[num_capsule, num_input_capsule])

        for i in range(routings):
            c = tf.nn.softmax(b, axis=1)
            outputs = squash(tf.linalg.matmul(c, inputs_tiled))
            if i < routings - 1:
                b += tf.linalg.matmul(outputs, inputs_tiled, transpose_b=True)

        return outputs
    return layer

# Model
input_shape = (23, 3)
inputs = Input(shape=input_shape)
reshaped_inputs = Reshape((23, 3, 1))(inputs)
primary_caps = PrimaryCaps(reshaped_inputs, dim_capsule=8, n_channels=16, kernel_size=(3, 3), strides=1, padding='valid')
caps_output = CapsuleLayer(num_capsule=31, dim_capsule=4)(primary_caps)
flatten_output = Flatten()(caps_output)
output = Dense(31*4, activation='linear')(flatten_output)
reshaped_output = Reshape((31, 4))(output)
model = Model(inputs=inputs, outputs=reshaped_output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
#Training the Model

In [12]:
history = model.fit(train_x, train_y, validation_split=0.2, epochs=10, batch_size=32)


Epoch 1/10
145/145 [==============================] - 46s 304ms/step - loss: 0.0346 - accuracy: 0.9543 - val_loss: 0.0419 - val_accuracy: 0.9394
Epoch 2/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0202 - accuracy: 0.9601 - val_loss: 0.0279 - val_accuracy: 0.9413
Epoch 3/10
145/145 [==============================] - 44s 300ms/step - loss: 0.0174 - accuracy: 0.9597 - val_loss: 0.0265 - val_accuracy: 0.9400
Epoch 4/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0172 - accuracy: 0.9603 - val_loss: 0.0269 - val_accuracy: 0.9449
Epoch 5/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0172 - accuracy: 0.9608 - val_loss: 0.0269 - val_accuracy: 0.9410
Epoch 6/10
145/145 [==============================] - 43s 298ms/step - loss: 0.0171 - accuracy: 0.9602 - val_loss: 0.0271 - val_accuracy: 0.9411
Epoch 7/10
145/145 [==============================] - 44s 301ms/step - loss: 0.0170 - accuracy: 0.9606 - val_loss: 0.0266 - val_ac

In [17]:
validation_x = np.load('coding_test/eval_x.npy')  
validation_y = np.load('coding_test/eval_y.npy')

validation_x = validation_x / np.max(validation_x)
validation_y = validation_y / np.max(validation_y)

validation_loss, validation_accuracy = model.evaluate(validation_x, validation_y)

print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")


35/35 [==============================] - 5s 135ms/step - loss: 0.0183 - accuracy: 0.9622
Validation Loss: 0.01833304390311241
Validation Accuracy: 0.9622111320495605


In [18]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 23, 3)]      0           []                               
                                                                                                  
 reshape_13 (Reshape)           (None, 23, 3, 1)     0           ['input_8[0][0]']                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 21, 1, 128)   1280        ['reshape_13[0][0]']             
                                                                                                  
 reshape_14 (Reshape)           (None, 336, 8)       0           ['conv2d_7[0][0]']               
                                                                                            

In [19]:
with open('model_summary.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))


In [24]:
!pip install keras-flops


In [25]:
from keras_flops import get_flops

# Calculate FLOPs
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {flops / 10 ** 9:.03f} G")


FLOPs: 0.032 G


In [28]:
!pip install --upgrade coremltools

In [30]:
!pip install tf2onnx

   ---------------------------------------- 0.0/454.7 kB ? eta -:--:--
   -- ------------------------------------- 30.7/454.7 kB 1.3 MB/s eta 0:00:01
   -------- ------------------------------- 92.2/454.7 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 256.0/454.7 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 454.7/454.7 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.3 MB 15.2 MB/s eta 0:00:01
   - -------------------------------------- 0.6/14.3 MB 8.0 MB/s eta 0:00:02
   - -------------------------------------- 0.6/14.3 MB 8.0 MB/s eta 0:00:02
   - -------------------------------------- 0.6/14.3 MB 8.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.7/14.3 MB 8.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.9/14.3 MB 7.3 MB/s eta 0:00:02
   ------ --------------------------------- 2.5/14.3 MB 7.9 MB/s eta 0:00:02
 

In [32]:
import tf2onnx
# Convert TensorFlow model to ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model)

onnx_model_path = 'model.onnx'
onnx.save_model(onnx_model, onnx_model_path)


ERROR:tf2onnx.tf_utils:pass1 convert failed for name: "model_3/conv2d_7/Conv2D"
op: "Conv2D"
input: "model_3/reshape_13/Reshape"
input: "model_3/conv2d_7/Conv2D/ReadVariableOp"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "data_format"
  value {
    s: "NHWC"
  }
}
attr {
  key: "dilations"
  value {
    list {
      i: 1
      i: 1
      i: 1
      i: 1
    }
  }
}
attr {
  key: "explicit_paddings"
  value {
    list {
    }
  }
}
attr {
  key: "padding"
  value {
    s: "VALID"
  }
}
attr {
  key: "strides"
  value {
    list {
      i: 1
      i: 1
      i: 1
      i: 1
    }
  }
}
attr {
  key: "use_cudnn_on_gpu"
  value {
    b: true
  }
}
, ex=Could not infer attribute `explicit_paddings` type from empty iterator


ValueError: Could not infer attribute `explicit_paddings` type from empty iterator

In [ ]:
# Convert ONNX model to CoreML
coreml_model = coremltools.convert(onnx_model_path)

coreml_model.save('my_model.mlmodel')
